In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.nn import convolution

from celluloid import Camera
from matplotlib import animation, rc
from IPython.display import HTML
from IPython.display import clear_output
from time import time
from scipy.special import factorial

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from FiniteDifferences.Maxwell3d import Maxwell3DFiniteDifference, DifferenceType

In [4]:
mwell = Maxwell3DFiniteDifference(mesh_size=0.01, step_size=1e-3, frame_rate=60,
                                 diff_type=DifferenceType.FORWARD_DIFFERENCE)

In [5]:
charge_density_callable = lambda x,y, z: 0
electric_field_callable = lambda x,y, z: np.zeros((3,))
magnetic_field_callable = lambda x, y, z: np.array([0,0,np.exp(-np.sum(np.square((np.array([x - 0.5, y- 0.5, z-0.5]).reshape((3,1,1)))/0.1), axis=0))[0,0]])

In [ ]:
B, E, I, p = mwell.evolve(magnetic_field_callable, 
                          electric_field_callable, 
                          charge_density_callable, 
                          integration_period=1,
                          order=10,
                          video=True)

In [ ]:
plt.imshow(np.max(B[:,:,:,1],axis=2))

In [ ]:
u=np.squeeze(convolution(B.reshape(1,100,100,100,3), filters=mwell._ampere_filter,padding="SAME").numpy())

In [ ]:
plt.imshow(np.max(u[:,:,:,1],axis=2))

In [ ]:
mwell._ampere_filter[:,:,1,0,2]

In [ ]:
mwell._faraday_filter[:,:,1,1,2]

In [ ]:
e=E[:, :, 50, :2]

In [ ]:
arrow_num = int(e.shape[0] // 20)
plt.quiver(np.linspace(0, 1, int(e.shape[0] / arrow_num)),
                              np.linspace(0, 1, int(e.shape[0] / arrow_num)),
                              e[::arrow_num, ::arrow_num, 0],
                              e[::arrow_num, ::arrow_num, 1],
                              scale=0.2)